In [1]:
from random import random, seed, randrange
from math import floor, pi
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time
import pandas as pd
from scipy.integrate import simps, trapz, cumtrapz
from scipy.ndimage.filters import gaussian_filter1d
from time import time
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 9]

In [2]:
%run -i "../../carbon_walls/gcmc.py"

In [3]:
filename = "../data/" + "y0.5rep2" + ".csv"
data_dump = pd.read_csv(filename).copy()

for i in range(data_dump.shape[0]):
    if np.isnan( data_dump.yco_sim[i]):
        yco = data_dump.yco_res[i]
        P_res = data_dump.P_res[i]*10**5 #[Pa]
        T = data_dump.T_res[i] + 273.15 #K
        fco, fme = PR_Fugacity( P_res/10**5, T, yco )
        fco = fco*10**5
        fme = fme*10**5
        
        sf = False
        mega_verbose = False

        s_box = 57.15
        N_max = 50000
        Vol = s_box**3
        kb = 1.3806*10**(7) #[Pa*A^3/K]
        Nco = 0 #floor(fco*Vol/kb/T)
        Nme = 0 #floor(fme*Vol/kb/T)
        Nc = 0
        rc = s_box
        beta = 1/T
        zz_co = beta*fco
        zz_me = beta*fme
        delta = 1
        pi_move = 0.5

        N_moves = 100
        N_equil = int( np.round( 200000/N_moves) )
        N_prod = int( np.round( 100000/N_moves) )

        t0 = time()
        rhocov,rhomev,Env,Pv,Ncov, Nmev = mc_run()
        t = time()
        
        Nco = Ncov.mean()
        Nme = Nmev.mean()
        data_dump.yco_sim[i] = Nco/(Nco + Nme)
        data_dump.P_sim[i] = Pv.mean()*10 #[bar]
        data_dump.E_sim[i] = Env.mean()
        data_dump.rhoco[i] = rhocov.mean()
        data_dump.rhome[i] = rhomev.mean()
        data_dump.yco_s[i] = (rhocov/(rhocov + rhomev)).std()
        data_dump.P_s[i] = Pv.std()*10 #[bar]
        data_dump.rhoco_s[i] = rhocov.std()
        data_dump.rhome_s[i] = rhomev.std()
        data_dump.E_s[i] = Env.std()
        data_dump.time[i] = t - t0
        print( i, data_dump.yco_sim[i],  data_dump.P_sim[i], data_dump.rhoco[i], data_dump.rhome[i], data_dump.time[i] )
        #filename = "../data_output/yco" + str(yco) + "P" + str(P) + "/run" + str(i) + ".csv"
        data_dump.to_csv( filename, index = False)

0 0.5028588551514851 9.955833197066749 0.00012289930934160175 0.00012150189406283784 33.9651198387146
1 0.49591649522958314 19.502896558923748 0.00024351370219649593 0.00024752401189236887 66.6874725818634
2 0.5017389165198993 29.335374728367807 0.00038407458558865795 0.0003814123498331138 100.3323962688446
3 0.4910220887692205 39.06636343860024 0.0005205990153401518 0.0005396364144855571 141.3026521205902
4 0.4855480640461422 47.06194782929803 0.0006375136649909302 0.0006754637973810473 176.02998566627502
5 0.48357702817758624 57.08519140340121 0.0007842868425440684 0.0008375578623207191 219.45490789413452
6 0.48283233674584 67.78754764951982 0.000988296669139486 0.0010585767358199654 276.0205190181732
7 0.49314682671589183 76.78445932589054 0.00117432369403477 0.0012069624268861965 314.254198551178
8 0.4804450627358375 84.16722358336119 0.0013270208858985923 0.0014350449335356402 356.3626048564911
9 0.4803107528278917 94.74268850193639 0.0015034507793867883 0.0016267118716365902 408.